In [1]:
"""LGBM Starter

This is watered-down version of one of my earlier scripts. 
Only very basic features are retained so hopefully it won't ruin the fun for you.
"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [2]:
order=pd.read_csv('../DataSets/jdata_user_order.csv',parse_dates=['o_date'])
order.head()

,﻿user_id,sku_id,o_id,o_date,o_area,o_sku_num
0,1,80036,480066,2017-03-09,14,1
1,1,16607,106933,2017-01-12,14,1
2,1,80036,542604,2017-04-14,14,1
3,1,80036,196335,2017-04-30,14,1
4,1,80036,42502,2017-03-22,14,2


In [3]:
order['user_id'] = order['\ufeffuser_id']
order = order.drop('\ufeffuser_id',axis=1)
order.head()

,sku_id,o_id,o_date,o_area,o_sku_num,user_id
0,80036,480066,2017-03-09,14,1,1
1,16607,106933,2017-01-12,14,1,1
2,80036,542604,2017-04-14,14,1,1
3,80036,196335,2017-04-30,14,1,1
4,80036,42502,2017-03-22,14,2,1


In [4]:
sku=pd.read_csv('../DataSets/jdata_sku_basic_info.csv')
sku['sku_id'] = sku['\ufeffsku_id']
sku = sku.drop('\ufeffsku_id',axis=1)
sku.head()
# sku.cate.value_counts()

,price,cate,para_1,para_2,para_3,sku_id
0,364.2,71,247.95,-1,-1,1
1,36.7,46,6.40,-1,-1,2
2,953.3,71,178.92,-1,-1,3
3,110.6,83,6366.88,-1,-1,4
4,241.5,71,31.74,-1,-1,5


In [5]:
user=pd.read_csv('../DataSets/jdata_user_basic_info.csv')
user['user_id'] = user['\ufeffuser_id']
user = user.drop('\ufeffuser_id',axis=1)
user.head()

,age,sex,user_lv_cd,user_id
0,3,2,3,1
1,3,2,3,2
2,3,0,5,3
3,3,2,3,4
4,4,2,2,5


In [6]:
action=pd.read_csv('../DataSets/jdata_user_action.csv',parse_dates=['a_date'])
action.head()

,﻿user_id,sku_id,a_date,a_num,a_type
0,1,80036,2017-04-14,4,1
1,1,96959,2017-01-12,2,1
2,1,8017,2017-03-09,1,1
3,1,80036,2017-04-30,6,1
4,1,16607,2017-01-12,3,1


In [7]:
action.columns[0]

'\ufeffuser_id'

In [8]:
action['user_id'] = action['\ufeffuser_id']
action = action.drop('\ufeffuser_id',axis=1)
action.head()

,sku_id,a_date,a_num,a_type,user_id
0,80036,2017-04-14,4,1,1
1,96959,2017-01-12,2,1,1
2,8017,2017-03-09,1,1,1
3,80036,2017-04-30,6,1,1
4,16607,2017-01-12,3,1,1


In [9]:
comment=pd.read_csv('../DataSets/jdata_user_comment_score.csv',parse_dates=['comment_create_tm'])
comment.head()

,﻿user_id,comment_create_tm,o_id,score_level
0,3,2017-03-21 20:38:25,403068,1
1,7,2016-12-12 09:31:43,194387,1
2,7,2016-12-17 16:37:23,197976,2
3,7,2016-12-12 09:30:50,327504,1
4,8,2017-02-16 22:26:59,173806,1


In [10]:
comment['user_id'] = comment['\ufeffuser_id']
comment = comment.drop('\ufeffuser_id',axis=1)
comment.head()

,comment_create_tm,o_id,score_level,user_id
0,2017-03-21 20:38:25,403068,1,3
1,2016-12-12 09:31:43,194387,1,7
2,2016-12-17 16:37:23,197976,2,7
3,2016-12-12 09:30:50,327504,1,7
4,2017-02-16 22:26:59,173806,1,8


In [11]:
comment.score_level.value_counts()

1    218250
2      3135
3      2899
Name: score_level, dtype: int64

In [12]:
# only 101 30
sku=pd.concat([sku[sku.cate==30],sku[sku.cate==101]],axis=0)

In [13]:
order=order.merge(comment,on=['o_id','user_id'],how='left')

In [14]:
order_=order.ix[:,[0,1,3,5]]

In [15]:
order_.head()

,sku_id,o_id,o_area,user_id
0,80036,480066,14,1
1,16607,106933,14,1
2,80036,542604,14,1
3,80036,196335,14,1
4,80036,42502,14,1


In [16]:
order_=order_.groupby(by=['user_id','sku_id','o_id']).sum()

In [17]:
order_.head()

o_area
user_id sku_id o_id          
1       16607  106933      14
               235938      14
        80036  42502       14
               196335      14
               480066      14

In [18]:
order_=order_.unstack(-1).fillna(0)
order_.columns=order_.columns.get_level_values(1)

IndexError: index 2093399605 is out of bounds for axis 0 with size 2093284757

In [ ]:
order_.head()

In [ ]:
order_.columns

In [ ]:
items = pd.read_csv(
    "jdata_sku_basic_info.csv",usecols=[0,1,2,3]).set_index("sku_id")

result = order_[
    pd.date_range("2017-01-01", periods=89+31)].copy()
del order

In [ ]:
items = items.reindex(result.index.get_level_values(1))
user=user.reindex(result.index.get_level_values(0))
result.head()
# 2017-05-01
# 2017-04-01

In [ ]:
def get_timespan(df, dt, minus, periods):
#     print(pd.date_range(dt - timedelta(days=minus), periods=periods))
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(result, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(result, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(result, t2017, 14, 14).mean(axis=1).values,
        "mean_29_2017": get_timespan(result, t2017, 29, 29).sum(axis=1).values
    })
    if is_train:
        y = result[
            pd.date_range(t2017, periods=30)
        ]
        y=y.sum(axis=1).values
        return X, y
    return X

In [ ]:
print("Preparing dataset...")
t2017 = date(2017, 2, 1)
X_l, y_l = [], []
for i in range(3):
    delta = timedelta(days=29 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

In [ ]:
del X_l, y_l
X_test = prepare_dataset(date(2017, 5, 1), is_train=False)

In [ ]:
X_test.shape

In [ ]:
items.index=range(len(items))
user.index=range(len(user))
i=pd.concat([items] * 3)
u=pd.concat([user] * 3)
i.shape,u.shape
i.index=range(len(i))
u.index=range(len(u))
X_train.index=range(len(X_train))
X_test.index=range(len(X_test))

In [ ]:
X_train=pd.concat([X_train,u,i],axis=1)
X_test=pd.concat([X_test,items,user],axis=1)

In [ ]:
X_train.head()

In [ ]:
X_train=pd.concat([X_train,pd.get_dummies(X_train['sex'],prefix='sex')],axis=1)
X_test=pd.concat([X_test,pd.get_dummies(X_test['sex'],prefix='sex')],axis=1)
X_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)

In [ ]:
X_train.drop(['user_id','cate','sex'],axis=1,inplace=True)
X_test.drop(['user_id','cate','sex'],axis=1,inplace=True)

In [ ]:
y_train=pd.DataFrame({'y':y_train})['y'].map(lambda x:1 if x>=1 else 0)

In [ ]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'binary',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 8
}

MAX_ROUNDS = 1000
val_pred = []
test_pred = []
cate_vars = []

dtrain = lgb.Dataset(
        X_train, label=y_train,
        categorical_feature=cate_vars,
#         weight=pd.concat([items["perishable"]] * 4) * 0.25 + 1
    )
# dval = lgb.Dataset(
#         X_val, label=y_val[:, i], reference=dtrain,
# #         weight=items["perishable"] * 0.25 + 1,
#         categorical_feature=cate_vars)
bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain], early_stopping_rounds=50, verbose_eval=50
    )
print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))

test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

In [ ]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame({'user':user.user_id.values,'y':list(y_test[:,0])})

In [ ]:
df_preds=df_preds.sort_values(by=['user','y'],ascending=[True,False])
df_preds.drop_duplicates(['user'],inplace=True)
df_preds=df_preds.sort_values(by=['y'],ascending=[False])

In [ ]:
df_preds.user.to_csv('submit.csv',index=False,header=None)

In [ ]:
df_preds.dropna(inplace=True)
df_preds.user=df_preds.user.map(lambda x:int(x))
df_preds['y']='2017-05-15'
df_preds.user=df_preds.user.astype(str)
df_preds.columns=['user_id','pred_date']

In [ ]:
df_preds[:50000].to_csv('submit.csv',index=False)